In [22]:
import pandas as pd

merged_merchant_with_categories = pd.read_parquet('../data/curated/merged_merchant_info_with_categories.parquet')

In [23]:
merged_merchant_with_categories.head()

,merchant_abn,category_label,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions
0,10023283211,Furniture and Home Furnishing,0.18,0.0,216.736529,155.285714
21,10142254217,"Media and Technology, Services",4.22,0.0,37.716001,144.571429
42,10165489824,Retail Goods,4.40,0.0,11236.094771,0.454545
53,10187291046,"Automotives, Retail Goods, Services",3.29,0.0,119.526004,16.800000
73,10192359162,Retail Goods,6.33,0.0,462.329450,19.250000


In [24]:
consumer_tbl = pd.read_csv('../data/raw/tbl_consumer.csv', delimiter='|')
userdf = pd.read_parquet('../data/raw/consumer_user_details.parquet')
merged = consumer_tbl.merge(userdf, on='consumer_id', how='left')

In [25]:
test_df = pd.read_parquet('../test/')

In [26]:
merged_with_test = merged.merge(test_df, on='user_id', how='left')
merged_with_test_dropped = merged_with_test.dropna()

In [27]:
# Dropped na since no one has bought from these merchants
merged_with_abn = merged_with_test_dropped.merge(merged_merchant_with_categories, on='merchant_abn', how='left')
merged_with_abn_dropped = merged_with_abn.dropna()

In [28]:
columns_to_drop = ['name', 'address', 'state', 'gender', 'dollar_value', 'order_id', 'order_datetime', 'category_label']

cleaned_merged_with_abn = merged_with_abn_dropped.drop(columns=columns_to_drop)

In [29]:
census_data = pd.read_csv('../data/curated/2021Census_G02_AUST_POA_curated.csv')
census_data = census_data.rename(columns={'POA_CODE_2021': 'postcode'})
census_data.head()
merged_with_census = cleaned_merged_with_abn.merge(census_data, on='postcode', how='left')

print(merged_with_census.head())
merged_with_census.isnull().sum()

   postcode  consumer_id  user_id  merchant_abn  take_rate_value  \
0      6935      1195503        1  2.800049e+10             4.24   
1      6935      1195503        1  8.369064e+10             3.15   
2      6935      1195503        1  9.668077e+10             5.91   
3      6935      1195503        1  4.989171e+10             5.80   
4      6935      1195503        1  6.240384e+10             6.46   

   fraud_probability  average_monthly_revenue  average_monthly_transactions  \
0           0.000000               245.258150                    180.523810   
1           0.000000                88.512653                   1707.238095   
2          29.555245               316.660874                   1482.571429   
3           0.000000                28.889602                  11786.952381   
4           0.000000                34.467794                    184.571429   

   Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
0                 NaN                   

postcode                               0
consumer_id                            0
user_id                                0
merchant_abn                           0
take_rate_value                        0
fraud_probability                      0
average_monthly_revenue                0
average_monthly_transactions           0
Median_age_persons               2244225
Median_mortgage_repay_monthly    2244225
Median_rent_weekly               2244225
Median_tot_hhd_inc_weekly        2244225
dtype: int64

In [30]:
merged_with_census_dropped = merged_with_census.dropna()
merged_with_census_dropped.head()

,postcode,consumer_id,user_id,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
553,2782,179208,2,8.077982e+10,4.71,0.0,34.567393,1735.142857,50.0,2080.0,430.0,1528.0
554,2782,179208,2,2.864764e+10,3.32,0.0,263.829390,23.476190,50.0,2080.0,430.0,1528.0
555,2782,179208,2,3.581349e+10,5.66,0.0,45.885892,182.761905,50.0,2080.0,430.0,1528.0
556,2782,179208,2,3.809009e+10,3.24,0.0,172.158074,2467.428571,50.0,2080.0,430.0,1528.0
557,2782,179208,2,3.897149e+10,3.61,0.0,166.642856,269.761905,50.0,2080.0,430.0,1528.0


In [31]:
aggregated_df = merged_with_census_dropped.groupby('merchant_abn').mean()
aggregated_df = aggregated_df.reset_index()
aggregated_df = aggregated_df.drop(columns=['consumer_id', 'postcode', 'user_id'])

aggregated_df.head()

,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053


In [32]:
merchant_categories = pd.read_parquet('../data/curated/merchant_categories.parquet')
merchant_categories = merchant_categories.drop(columns=['category', 'revenue_rate', 'fraud_probability', 'take_rate_value'])
category_df = aggregated_df.merge(merchant_categories, on='merchant_abn', how='left')



In [33]:
merchant_fraud = pd.read_csv('../data/raw/merchant_fraud_probability.csv')
final_df = category_df.merge(merchant_fraud, on='merchant_abn', how='left')
final_df['fraud_probability_y'] = final_df['fraud_probability_y'].apply(lambda x: 0 if pd.isna(x) else 1)
final_df = final_df.drop(columns = ['order_datetime'])
final_df = final_df.rename(columns={'fraud_probability_x': 'consumer_fraud', 'fraud_probability_y': 'merchant_fraud'})


In [34]:
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [35]:
final_df['Median_age_persons'] = (25 - final_df['Median_age_persons']).abs()
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,18.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,18.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,14.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,18.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,18.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [36]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming final_df is your DataFrame

# Columns to exclude from standardization
exclude_columns = ['merchant_abn', 'consumer_fraud', 'merchant_fraud', 'category_label', 'name']

# Columns to standardize
columns_to_standardize = [col for col in final_df.columns if col not in exclude_columns]

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply the scaler to the columns to be standardized
final_df[columns_to_standardize] = scaler.fit_transform(final_df[columns_to_standardize])

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,-2.372505,0.0,-0.330774,-0.011416,-0.069014,-0.083224,-0.097389,-0.064340,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,-0.098393,0.0,-0.388340,-0.027317,-0.139663,0.045873,-0.024827,0.068302,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,0.002929,0.0,3.212584,-0.241195,-3.404137,1.037388,1.625141,3.051869,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,-0.621889,0.0,-0.362033,-0.216937,0.337422,0.026400,-0.265919,-0.090710,Ultricies Dignissim Lacus Foundation,"Retail Goods, Services",0
4,1.019236e+10,1.089324,0.0,-0.251802,-0.213301,-0.137805,0.213058,0.479304,0.654616,Enim Condimentum PC,Retail Goods,0


In [37]:
#final_df['Median_age_persons'] = (1 - final_df['Median_age_persons'])
#final_df.head()

In [65]:
import pandas as pd

# Assuming final_df is your DataFrame

# Define the weights for each column
weights = {
    'merchant_fraud': -7.5,
    'average_monthly_revenue': 1.5,
    'take_rate_value': 0.5,
    'consumer_fraud': -0.6,
    'average_monthly_transactions': 0.9,
    'Median_tot_hhd_inc_weekly': 0.2,
    'Median_age_persons': -0.15,
    'Median_mortgage_repay_monthly': -0.1,
    'Median_rent_weekly': -0.1
}

# Calculate the score for each row
final_df['score'] = (
    final_df['merchant_fraud'] * weights['merchant_fraud'] +
    final_df['average_monthly_revenue'] * weights['average_monthly_revenue'] +
    final_df['take_rate_value'] * weights['take_rate_value'] +
    final_df['consumer_fraud'] * weights['consumer_fraud'] +
    final_df['average_monthly_transactions'] * weights['average_monthly_transactions'] +
    final_df['Median_tot_hhd_inc_weekly'] * weights['Median_tot_hhd_inc_weekly'] +
    final_df['Median_age_persons'] * weights['Median_age_persons'] +
    final_df['Median_mortgage_repay_monthly'] * weights['Median_mortgage_repay_monthly'] +
    final_df['Median_rent_weekly'] * weights['Median_rent_weekly']
)

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,score
0,1.002328e+10,-2.372505,0.0,-0.330774,-0.011416,-0.069014,-0.083224,-0.097389,-0.064340,Felis Limited,Furniture and Home Furnishing,0,-1.677143
1,1.014225e+10,-0.098393,0.0,-0.388340,-0.027317,-0.139663,0.045873,-0.024827,0.068302,Arcu Ac Orci Corporation,"Media and Technology, Services",0,-0.623786
2,1.016549e+10,0.002929,0.0,3.212584,-0.241195,-3.404137,1.037388,1.625141,3.051869,Nunc Sed Company,Retail Goods,0,5.458007
3,1.018729e+10,-0.621889,0.0,-0.362033,-0.216937,0.337422,0.026400,-0.265919,-0.090710,Ultricies Dignissim Lacus Foundation,"Retail Goods, Services",0,-1.094041
4,1.019236e+10,1.089324,0.0,-0.251802,-0.213301,-0.137805,0.213058,0.479304,0.654616,Enim Condimentum PC,Retail Goods,0,0.057346


In [66]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Sort the DataFrame by the 'score' column in descending order
sorted_df = final_df.sort_values(by='score', ascending=False)

# Select the top 100 merchants
top_100_merchants = sorted_df.head(100)

# Display the top 100 merchants
top_100_merchants.head(30)

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,score
1372,3.915015e+10,1.460837,0.0,11.847643,-0.240385,-6.422190,-0.887322,-2.439787,2.043061,Aliquam Eu Institute,Services,0,19.990188
895,2.906845e+10,-0.627518,0.0,11.481340,-0.240385,15.997635,-9.168425,-14.757751,13.882157,Phasellus LLP,Services,0,19.461308
1731,4.704774e+10,-1.606963,0.0,12.750036,-0.240385,-14.182899,1.859322,3.226476,-3.785603,Aenean Gravida Institute,Services,0,18.966960
692,2.485245e+10,-0.818904,0.0,-0.390804,20.217869,-0.042681,-0.015155,0.005401,-0.000977,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0,17.207607
3478,8.657848e+10,1.145614,0.0,-0.389202,19.027866,-0.017165,0.012455,0.053073,0.038967,Leo In Consulting,"Retail Goods, Services",0,17.117899
2467,6.420342e+10,-0.863936,0.0,-0.391159,18.187323,-0.024466,0.018950,0.060854,0.046309,Pede Nonummy Corp.,Furniture and Home Furnishing,0,15.354836
1854,4.989171e+10,0.790987,0.0,-0.391178,17.250669,-0.031752,-0.000536,0.027777,0.015705,Non Vestibulum Industries,Furniture and Home Furnishing,0,15.339509
1662,4.562922e+10,1.455208,0.0,-0.388702,15.888797,-0.044448,-0.013362,0.021641,0.022286,Lacus Consulting,"Automotives, Retail Goods",0,14.454764
1719,4.680414e+10,-0.824533,0.0,-0.390783,16.325464,-0.015284,0.010812,0.023865,0.019440,Suspendisse Dui Corporation,Services,0,13.697190
3625,8.972601e+10,0.909196,0.0,-0.387178,15.020128,-0.050592,-0.000490,0.036045,0.032474,Est Nunc Consulting,Furniture and Home Furnishing,0,13.402474


In [67]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Define the category labels
categories = ['Retail Goods', 'Media and Technology', 'Furniture and Home Furnishing', 'Services', 'Automotives']

# Initialize variables to store the top 10 merchants for each category
top_10_retail_goods = None
top_10_media_technology = None
top_10_furniture_home_furnishing = None
top_10_services = None
top_10_automotives = None

# Loop through each category and get the top 10 merchants
for category in categories:
    # Filter and sort the DataFrame by the current category and score
    top_10_merchants = final_df[final_df['category_label'].apply(lambda x: category in x)].sort_values(by='score', ascending=False).head(10)
    print(top_10_merchants.head(10))
    top_10_merchants = top_10_merchants[['name']].reset_index(drop=True)
    top_10_merchants.index = pd.RangeIndex(start=1, stop=len(top_10_merchants) + 1, step=1)


    # Store the result in the corresponding variable
    if category == 'Retail Goods':
        top_10_retail_goods = top_10_merchants
    elif category == 'Media and Technology':
        top_10_media_technology = top_10_merchants
    elif category == 'Furniture and Home Furnishing':
        top_10_furniture_home_furnishing = top_10_merchants
    elif category == 'Services':
        top_10_services = top_10_merchants
    elif category == 'Automotives':
        top_10_automotives = top_10_merchants



      merchant_abn  take_rate_value  consumer_fraud  average_monthly_revenue  \
692   2.485245e+10        -0.818904        0.000000                -0.390804   
3478  8.657848e+10         1.145614        0.000000                -0.389202   
1662  4.562922e+10         1.455208        0.000000                -0.388702   
1558  4.318652e+10         0.042331        0.000000                -0.386058   
2419  6.329052e+10         1.173759        0.000000                -0.392708   
3206  8.032405e+10        -1.646366        0.000000                -0.388656   
2463  6.414355e+10         0.836019        0.000000                 6.278965   
2392  6.268859e+10        -1.978476        0.000000                 6.388440   
1363  3.891866e+10        -0.453020        0.000000                 5.246069   
2604  6.726425e+10         1.128727       18.210891                16.280861   

      average_monthly_transactions  Median_age_persons  \
692                      20.217869           -0.042681   
347

In [68]:
# Display the top 10 merchants for each category
print("Top 10 merchants in 'Retail Goods':")
print(top_10_retail_goods)
print("\n")

print("Top 10 merchants in 'Media and Technology':")
print(top_10_media_technology)
print("\n")

print("Top 10 merchants in 'Furniture and Home Furnishing':")
print(top_10_furniture_home_furnishing)
print("\n")

print("Top 10 merchants in 'Services':")
print(top_10_services)
print("\n")

print("Top 10 merchants in 'Automotives':")
print(top_10_automotives)
print("\n")

Top 10 merchants in 'Retail Goods':
                                 name
1                      Erat Vitae LLP
2                   Leo In Consulting
3                    Lacus Consulting
4      Lorem Ipsum Sodales Industries
5   Vehicula Pellentesque Corporation
6         Ipsum Dolor Sit Corporation
7            Fusce Feugiat Associates
8            Lobortis Nisi Associates
9                 Faucibus Foundation
10          Elit Dictum Eu Foundation


Top 10 merchants in 'Media and Technology':
                                 name
1     Placerat Eget Venenatis Limited
2                   Nullam Consulting
3           Suspendisse Ac Associates
4                  Vel Est Tempor LLP
5                   Ac Eleifend Corp.
6                        Eros Limited
7   Posuere Cubilia Curae Corporation
8                     Eu Placerat LLC
9                        At Sem Corp.
10      Semper Cursus Integer Limited


Top 10 merchants in 'Furniture and Home Furnishing':
                           